In [1]:
from model.inception_resnet_v1 import InceptionResnetV1
from model.mtcnn import MTCNN
import torch
from PIL import Image

### Conversion for FaceNet

In [4]:
resnet = InceptionResnetV1(pretrained='vggface2', classify=False).cuda()
# resnet.load_state_dict(torch.load('tamer_exp6_b256_e5_i5000_id64_f15_s12.pt'), strict=False)
resnet.eval()
example = torch.rand(1, 3, 160, 160).cuda()

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(resnet, example)


In [5]:
traced_script_module.save("model.pt")

### Verify Orig Model vs Traced Model

In [4]:
mtcnn = MTCNN(image_size=160, margin=0, device='cuda', keep_all=True)

In [29]:
img = Image.open('067914.jpg')

In [33]:
img = Image.open('085791.jpg')

In [34]:
img_tensor, prob, = mtcnn(img, return_prob=True)

In [35]:
orig_op = resnet(img_tensor.to('cuda'))

In [10]:
traced_op = traced_script_module(img_tensor.to('cuda'))

In [32]:
op1 = orig_op

In [36]:
op2 = orig_op

In [37]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
cos(op1, op2)

tensor([0.8422], device='cuda:0', grad_fn=<DivBackward0>)

In [6]:
traced_script_module(example).size()

torch.Size([1, 512])

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

### Conversion for MTCNN

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

In [2]:
mtcnn = MTCNN(image_size=160, margin=0, device='cuda:0', keep_all=True)

In [3]:
mtcnn.eval()

MTCNN(
  (pnet): PNet(
    (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=10)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(10, 16, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=16)
    (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (prelu3): PReLU(num_parameters=32)
    (conv4_1): Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1))
    (softmax4_1): Softmax(dim=1)
    (conv4_2): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
  )
  (rnet): RNet(
    (conv1): Conv2d(3, 28, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=28)
    (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(28, 48, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=48)
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv3): Conv2d(48, 64,

In [6]:
img = Image.open('data/exp6/test/000130.jpg')

In [9]:
arr = np.asarray(img)

In [16]:
example = np.array([arr])

In [19]:
eg = torch.rand(example.shape).cuda()

In [21]:
eg.size()

torch.Size([1, 232, 204, 3])

In [22]:
traced_script_module = torch.jit.trace(mtcnn, eg)

/home/dh/Desktop/Github/facenet_aip/src/model/utils/detect_face.py:31: TracerWarning: torch.as_tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  imgs = torch.as_tensor(imgs, device=device)
/home/dh/Desktop/Github/facenet_aip/src/model/utils/detect_face.py:48: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  batch_size = len(imgs)
/home/dh/Desktop/Github/facenet_aip/src/model/utils/detect_face.py:51: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in 

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'Tensor'